<a href="https://colab.research.google.com/github/DiegoDelgado06/PROGCOM-B/blob/main/HARCORE-QUEST-THREE-HQ3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import json
import os
from datetime import datetime
from IPython.display import display, clear_output
import ipywidgets as widgets
import time

STATE_FILE = "dad_state.json"

class Dad:
    def __init__(self):
        self.max_value = 100
        self.load_state()

    def load_state(self):
        if os.path.exists(STATE_FILE):
            with open(STATE_FILE, "r") as f:
                data = json.load(f)
                self.hunger = data["hunger"]
                self.energy = data["energy"]
                self.happiness = data["happiness"]
                self.last_update = datetime.fromisoformat(data["last_update"])
                self.update_state_based_on_time()
        else:
            self.hunger = 50
            self.energy = 50
            self.happiness = 50
            self.last_update = datetime.now()
            self.save_state()

    def save_state(self):
        data = {
            "hunger": self.hunger,
            "energy": self.energy,
            "happiness": self.happiness,
            "last_update": datetime.now().isoformat()
        }
        with open(STATE_FILE, "w") as f:
            json.dump(data, f)

    def update_state_based_on_time(self):
        now = datetime.now()
        delta = now - self.last_update
        minutes_passed = delta.total_seconds() // 60
        self.hunger = max(0, self.hunger - int(minutes_passed * 0.5))
        self.energy = max(0, self.energy - int(minutes_passed * 0.3))
        self.happiness = max(0, self.happiness - int(minutes_passed * 0.4))
        self.last_update = now

    def feed(self):
        self.hunger = min(self.max_value, self.hunger + 20)

    def rest(self):
        self.energy = min(self.max_value, self.energy + 20)

    def play(self):
        self.happiness = min(self.max_value, self.happiness + 20)

dad = Dad()

# Widgets
hunger_bar = widgets.FloatProgress(min=0, max=100, description='Hambre:', bar_style='success')
energy_bar = widgets.FloatProgress(min=0, max=100, description='Energía:', bar_style='success')
happiness_bar = widgets.FloatProgress(min=0, max=100, description='Felicidad:', bar_style='success')

feed_button = widgets.Button(description="🍗 Dar de Comer")
rest_button = widgets.Button(description="😴 Descansar")
play_button = widgets.Button(description="🎮 Jugar")

status_label = widgets.Label(value="")

def update_bars():
    dad.update_state_based_on_time()
    hunger_bar.value = dad.hunger
    energy_bar.value = dad.energy
    happiness_bar.value = dad.happiness

    def style(bar, val):
        if val > 70:
            bar.bar_style = 'success'
        elif val > 40:
            bar.bar_style = 'warning'
        else:
            bar.bar_style = 'danger'

    style(hunger_bar, dad.hunger)
    style(energy_bar, dad.energy)
    style(happiness_bar, dad.happiness)

    dad.save_state()

def on_feed_clicked(b):
    dad.feed()
    update_bars()
    status_label.value = "Le diste de comer a Dad 🐔"

def on_rest_clicked(b):
    dad.rest()
    update_bars()
    status_label.value = "Dad está descansando 😴"

def on_play_clicked(b):
    dad.play()
    update_bars()
    status_label.value = "Jugaste con Dad 🎮"

feed_button.on_click(on_feed_clicked)
rest_button.on_click(on_rest_clicked)
play_button.on_click(on_play_clicked)

def show_ui():
    update_bars()
    ui = widgets.VBox([
        hunger_bar,
        energy_bar,
        happiness_bar,
        widgets.HBox([feed_button, rest_button, play_button]),
        status_label
    ])
    display(ui)

show_ui()
